In [1]:
require('nn')
require('hdf5')
require('optim')

f = hdf5.open("data.hdf5", "r")



train_rnn_X = f:read('train_rnn_X'):all()
train_rnn_Y = f:read('train_rnn_Y'):all()
train_X_seqeuence =  f:read('train_X_sequence'):all()
train_Y  = f:read('train_Y'):all()
valid_reduced_X = f:read('valid_reduced_X'):all()
valid_reduced_Y = f:read('valid_reduced_Y'):all()
windows_train = f:read('windows_train'):all()
windows_valid = f:read('windows_valid'):all()
test = f:read('test'):all()
valid_kaggle_with_spaces = f:read('valid_kaggle_with_spaces'):all()
valid_kaggle_without_spaces = f:read('valid_kaggle_without_spaces'):all()
valid_answers = f:read('valid_answers'):all()
nfeatures = (f:read('nfeatures'):all())[1]
nclasses = (f:read('nclasses'):all())[1]




f:close()

window_size = windows_train:size(2)
n_actual = nclasses

In [37]:
valid_reduced_Y:size()

 115716
[torch.LongStorage of size 1]



In [2]:

embedded_size = 15
D_h = 100

lookup1 = nn.LookupTable(nfeatures*window_size,embedded_size)
linear1 = nn.Linear(window_size*embedded_size, D_h)
linear2 = nn.Linear(D_h, n_actual)
h = nn.Sequential()
h:add(lookup1)
--lookup1.weight:renorm(2, 2, 1)
h:add(nn.Reshape(window_size*embedded_size, True))
h:add(linear1)
h:add(nn.Tanh())
h:add(linear2)
h:add(nn.LogSoftMax())
mse = nn.ClassNLLCriterion()
--mse = nn.CrossEntropyCriterion()
w, dl_dw = h:getParameters()

In [3]:
X_set = windows_train
Y_set = train_Y

In [4]:
function feval(w_new)
   
    bsize= batch
    local idx = torch.randperm(X_set:size(1)):sub(1,bsize)

    local x = torch.Tensor(bsize, X_set:size(2))

    
    --y = y:reshape(Y_train:size(1), 1)
    
    local y_ = torch.Tensor(bsize, 1)

    for i=1,bsize do
        x[i] = X_set[idx[i]]
        y_[i] = Y_set[idx[i]]
    end
    
    y_ = y_:squeeze()
    
    local inputs = x
    local targets = y_
    -- reset gradients (gradients are always accumulated, to accommodate
    -- batch methods)
    dl_dw:zero()
    lookup1.weight:renorm(2, 1, 5)
    -- evaluate the loss function and its derivative with respect to x, given a mini batch
    local prediction = h:forward(inputs)
    local loss_w = mse:forward(prediction, targets)
    h:backward(inputs, mse:backward(prediction, targets))
    
    return loss_w, dl_dw
            
end

batch = 10000

-- cycle on data
for i = 1,1000 do
    -- train a mini_batch of batchSize in parallel
    _, fs = optim.adam(feval,w)

    if i % 10 == 0 then
        print('loss for iteration ' .. i  .. ' is ' .. fs[1] )
        -- print(sgd_params)
    end
end

loss for iteration 10 is 0.54082166820954	


loss for iteration 20 is 0.45449875638635	


loss for iteration 30 is 0.4087458622284	


loss for iteration 40 is 0.38191122489268	


loss for iteration 50 is 0.36639082297053	


...

loss for iteration 960 is 0.15362759280672	


loss for iteration 970 is 0.15909139220473	


loss for iteration 980 is 0.159249930162	


loss for iteration 990 is 0.16240401304933	


loss for iteration 1000 is 0.15328034213911	


In [83]:
function perplexity(input_set, output_set)
    sum = 0
    distributions = torch.Tensor(input_set:size(1),2)
    for i = 1,input_set:size(1) do
        a = (h:forward(input_set[i]))
        
        distribution = (nn.LogSoftMax():forward(a))
        distributions[i] = torch.exp(distribution)
        --print((nn.LogSoftMax():forward(a)))
        answer = distribution[output_set[i]]

        sum = sum + answer
        
    end
    return torch.exp(-sum/input_set:size(1)), distributions
end


In [84]:
perp, dist = perplexity(windows_valid:sub(1,1000), valid_reduced_Y)
print(perp)

1.1866876057044	


In [40]:
torch.log(perp)

0.14645153348794	


In [ ]:
distributions:size()

In [ ]:
function write2file(scores, fname)
    f = io.open(fname, "w")
    f:write("ID,Class1,CLass2,Class3,Class4,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,CLass13,Class14,Class15,Class16,Class17,Class18,Class19,Class20,Class21,Class22,Class23,CLass24,Class25,Class26,Class27,Class28,Class29,Class30,Class31,Class32,Class33,Class34,CLass35,Class36,Class37,Class38,Class39,Class40,Class41,Class42,Class43,Class44,Class45,CLass46,Class47,Class48,Class49,Class50\n")
    for i=1,scores:size(1) do
        s = tostring(i)
        for j=1, scores:size(2) do
            s = s .. "," .. tostring(scores[i][j])
        end
        f:write(s .. "\n")
    end
    f:close()
end

In [ ]:
write2file(distributions, 'predictions.txt')

In [ ]:
nn.Index()